In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forest-cover-type-prediction/train.csv.zip
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv.zip
/kaggle/input/forest-cover-type-prediction/test3.csv
/kaggle/input/forest-cover-type-prediction/train.csv
/kaggle/input/forest-cover-type-prediction/test.csv
/kaggle/input/forest-cover-type-prediction/test.csv.zip


In [3]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [5]:
traindata=pd.read_csv("/kaggle/input/forest-cover-type-prediction/train.csv")
testdata=pd.read_csv("/kaggle/input/forest-cover-type-prediction/test.csv")

In [6]:
testdata.head(5)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,...,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,...,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,...,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,...,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,...,0,0,0,0,0,0,0,0,0,0


In [7]:
traindata.isna().sum()

Id                                    0
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0


In [10]:
traindata.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [11]:
traindata.drop(['Id'],axis=1)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,2607,243,23,258,7,660,170,251,214,1282,...,0,0,0,0,0,0,0,0,0,3
15116,2603,121,19,633,195,618,249,221,91,1325,...,0,0,0,0,0,0,0,0,0,3
15117,2492,134,25,365,117,335,250,220,83,1187,...,0,0,0,0,0,0,0,0,0,3
15118,2487,167,28,218,101,242,229,237,119,932,...,0,0,0,0,0,0,0,0,0,3


In [12]:
X=traindata.drop(['Cover_Type'],axis=1)
y=traindata['Cover_Type']

In [14]:
X.shape

(15120, 55)

In [15]:
y.shape

(15120,)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:

RandomForestClassifierModel = RandomForestClassifier(criterion = 'gini',n_estimators=100,max_depth=2,random_state=33) #criterion can be also : entropy
RandomForestClassifierModel.fit(x_train, y_train)
#Calculating Prediction
RFC_y_pred = RandomForestClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, RFC_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, RFC_y_pred, average='macro')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, RFC_y_pred, average='macro')
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, RFC_y_pred, average='macro')
print(f"F1-Score: {f1}")

Accuracy Score: 0.5982142857142857
Precision: 0.6049202149797516
Recall: 0.5950598252175584
F1-Score: 0.5764977090846656


In [19]:
DecisionTreeClassifierModel = DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=33) #criterion can be entropy
DecisionTreeClassifierModel.fit(x_train, y_train)

#Calculating Prediction
DTC_y_pred = DecisionTreeClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, DTC_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, DTC_y_pred, average='macro')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, DTC_y_pred, average='macro')
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, DTC_y_pred, average='macro')
print(f"F1-Score: {f1}")

Accuracy Score: 0.5846560846560847
Precision: 0.4964270709702773
Recall: 0.5846981534994807
F1-Score: 0.5323778009124251


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:

KNNClassifierModel = KNeighborsClassifier(n_neighbors= 5,weights ='uniform', # it can be distance
                                          algorithm='auto') # it can be ball_tree, kd_tree,brute
KNNClassifierModel.fit(x_train, y_train)

#Calculating Prediction
KNN_y_pred = KNNClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, KNN_y_pred)
print(f"Accuracy Score: {val_accuracy}")
# Calculate precision
precision = precision_score(y_test, KNN_y_pred, average='macro')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, KNN_y_pred, average='macro')
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, KNN_y_pred, average='macro')
print(f"F1-Score: {f1}")

Accuracy Score: 0.8128306878306878
Precision: 0.8074608428640854
Recall: 0.8120950981256077
F1-Score: 0.8062980793803154


In [23]:
# Initializing CatBoostClassifier
CatBoostClassifierModel = CatBoostClassifier(logging_level='Silent')

# Fitting the model
CatBoostClassifierModel.fit(x_train, y_train)

# Calculating Prediction
CatBoost_y_pred = CatBoostClassifierModel.predict(x_test)

# Calculate accuracy
val_accuracy = accuracy_score(y_test, CatBoost_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, CatBoost_y_pred, average='macro')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, CatBoost_y_pred, average='macro')
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, CatBoost_y_pred, average='macro')
print(f"F1-Score: {f1}")

Accuracy Score: 0.8707010582010583
Precision: 0.8674576480008065
Recall: 0.8701978055347307
F1-Score: 0.8679680814293649


In [31]:
test_predictions = CatBoostClassifierModel.predict(testdata).flatten()
# Create submission file
submission = pd.DataFrame({'Id': testdata['Id'], 'Cover_Type': test_predictions})
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created successfully!")


Submission file created successfully!
